In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'item_categories.csv', 'sales_train.csv', 'sample_submission.csv', 'items.csv', 'shops.csv']


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from itertools import product

import gc

def downcast_dtypes(df, i=16):
    '''
        Changes column types in the dataframe: 
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    #df[float_cols] = df[float_cols].astype(np.float32)
    #df[int_cols]   = df[int_cols].astype(np.int32)
    
    for fltc in float_cols:
        if i == 16:
            df[fltc] = df[fltc].astype(np.float16)
        else:
            df[fltc] = pd.to_numeric(df[fltc], downcast='float')
    for intc in int_cols:
        df[intc] = pd.to_numeric(df[intc], downcast='integer')
    
    return df

In [3]:
sales = pd.read_csv('../input/sales_train.csv')
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
items = pd.read_csv('../input/items.csv')
item_categories = pd.read_csv('../input/item_categories.csv')
shops = pd.read_csv('../input/shops.csv')
test = pd.read_csv('../input/test.csv')

sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')

In [5]:
test.head(1)

,ID,shop_id,item_id
0,0,5,5037


* item_price

In [6]:
# does item_price is different for shops, for same item
#pd.pivot_table(sales, index=['date_block_num', 'shop_id', 'item_id', 'item_price']).head(6)
# item 51 has 127 & 130 as price for shop & date 0

## Outliers:

* ### Boxplot

In [7]:
#sns.boxplot(sales.item_cnt_day)

In [8]:
#sns.boxplot(sales.item_price)

In [9]:
# remove the outliers
sales = sales[sales.item_price < 100000] 
sales = sales[sales.item_cnt_day < 1001]

* item with -ve price

In [10]:
#sum(sales.item_price < 0)

In [11]:
#sales[(sales.shop_id==32)&(sales.item_id==2973)&(sales.date_block_num==4)]#.item_price

In [12]:
mdn = sales[(sales.shop_id == 32)
            &(sales.item_id == 2973)
            &(sales.date_block_num == 4)
            &(sales.item_price > 0)].item_price.median()

sales.loc[sales.item_price < 0, 'item_price'] = mdn

* Duplicate shops

In [13]:
# due to same name, taken from a kaggle kernel(https://www.kaggle.com/dlarionov/feature-engineering-xgboost)
# Якутск Орджоникидзе, 56
sales.loc[sales.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales.loc[sales.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales.loc[sales.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

* Frome Russian guy's kernel
 * Each shop_name starts with the city name.

In [14]:
# geting city's names from the shop names & encoding the labels
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_id'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_id']]

shops.head()

,shop_id,city_id
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


 * Each category contains type and subtype in its name.
 

In [15]:
# item categories names contain further types and subtypes
item_categories['split'] = item_categories['item_category_name'].str.split('-')
item_categories['type'] = item_categories['split'].map(lambda x: x[0].strip())
item_categories['type_id'] = LabelEncoder().fit_transform(item_categories['type'])
# if subtype is nan then type
item_categories['subtype'] = item_categories['split'].map(lambda x: 
                                                          x[1].strip() 
                                                          if len(x) > 1 
                                                          else x[0].strip())
item_categories['subtype_id'] = LabelEncoder().fit_transform(item_categories['subtype'])
item_categories = item_categories[['item_category_id','type_id', 'subtype_id']]

items.drop(['item_name'], axis=1, inplace=True)
item_categories.head()

,item_category_id,type_id,subtype_id
0,0,0,29
1,1,1,9
2,2,1,10
3,3,1,11
4,4,1,13


* revenue as feature

In [16]:
sales['revenue'] = sales['item_price'] * sales['item_cnt_day']

## Grid:

- prediction are need to be made based on monthly data

In [17]:
# create grid of shop and items and add sum() of their item_cnt_day (0 for NaN)
index_cols = ['date_block_num', 'shop_id', 'item_id']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[[block_num], cur_shops, cur_items])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

* add item_category_id to sales, grid & test

In [18]:
sales = pd.merge(sales, items, how='left', on=['item_id'])
grid  = pd.merge(grid,  items, how='left', on=['item_id'])
test  = pd.merge(test,  items, how='left', on=['item_id'])

* add type_id, subtype_id, city_id

In [19]:
sales = pd.merge(sales, item_categories, how='left', on=['item_category_id'])
grid  = pd.merge(grid,  item_categories, how='left', on=['item_category_id'])
test  = pd.merge(test,  item_categories, how='left', on=['item_category_id'])

In [20]:
sales = pd.merge(sales, shops, how='left', on=['shop_id'])
grid  = pd.merge(grid,  shops, how='left', on=['shop_id'])
test  = pd.merge(test,  shops, how='left', on=['shop_id'])

In [21]:
sales.sample()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,item_category_id,type_id,subtype_id,city_id
1447340,2014-02-16,13,41,10904,199.0,1.0,199.0,55,13,2,18


!!!!!!!! price, cnt, revenue not in all_data (and not in test too, create lag features and drop them)

In [22]:
grid.head()

,date_block_num,shop_id,item_id,item_category_id,type_id,subtype_id,city_id
0,0,59,22154,37,11,1,30
1,0,59,2552,58,13,27,30
2,0,59,2554,58,13,27,30
3,0,59,2555,56,13,3,30
4,0,59,2564,59,13,40,30


In [23]:
test.head()

,ID,shop_id,item_id,item_category_id,type_id,subtype_id,city_id
0,0,5,5037,19,5,10,3
1,1,5,5320,55,13,2,3
2,2,5,5233,19,5,10,3
3,3,5,5232,23,5,16,3
4,4,5,5268,20,5,11,3


In [24]:
#(sales.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).item_cnt_day.sum() > 20).item_cnt_day.sum()

* 11,463 enteries outside the range of (0, 20)

In [25]:
#(sales.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).item_cnt_day.sum().clip(0,20) > 20).item_cnt_day.sum()

## target, target_shop, target_item, target_item_cat, ... :

In [26]:
# add sum of sales in a month for shops & items, target
#####################
#get aggregated values for (shop_id, item_id, month)
#gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target': 'sum'}})
#fix column names
#gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]

# alternative way to sum item_cnt_day, without warning from pandas for using dict in agg()
si = sales.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).item_cnt_day.sum()#.clip(0,20)
si = si.rename(columns={'item_cnt_day': 'target'})
#####################

# add column to the data
all_data = pd.merge(grid, si, how='left', on=index_cols).fillna(0)

#### target_shop
# add sum of sales in a month for shops, target_shop
s = sales.groupby(['date_block_num', 'shop_id'], as_index=False).item_cnt_day.sum()
s = s.rename(columns={'item_cnt_day' : 'target_shop'})

all_data = pd.merge(all_data, s, how='left', on=['date_block_num', 'shop_id']).fillna(0)

#### target_item
# add sum of sales in a month for items, target_item
i = sales.groupby(['date_block_num', 'item_id'], as_index=False).item_cnt_day.sum()
i = i.rename(columns={'item_cnt_day' : 'target_item'})

all_data = pd.merge(all_data, i, how='left', on=['date_block_num', 'item_id']).fillna(0)

#### target_item_cat
# add sum of sales in a month for item_category, target_item_cat
ic = sales.groupby(['date_block_num', 'item_category_id'], as_index=False).item_cnt_day.sum()
ic = ic.rename(columns={'item_cnt_day' : 'target_item_cat'})

all_data = pd.merge(all_data, ic, how='left', on=['date_block_num', 'item_category_id']).fillna(0)

#### target_item_type
# add sum of sales in a month for type_id, target_item_type
it = sales.groupby(['date_block_num', 'type_id'], as_index=False).item_cnt_day.sum()
it = it.rename(columns={'item_cnt_day' : 'target_item_type'})

all_data = pd.merge(all_data, it, how='left', on=['date_block_num', 'type_id']).fillna(0)

#### target_item_subtype
# add sum of sales in a month for subtype_id, target_item_subtype
ist = sales.groupby(['date_block_num', 'subtype_id'], as_index=False).item_cnt_day.sum()
ist = ist.rename(columns={'item_cnt_day' : 'target_item_subtype'})

all_data = pd.merge(all_data, ist, how='left', on=['date_block_num', 'subtype_id']).fillna(0)

#### target_city
# add sum of sales in a month for city_id, target_city
ci = sales.groupby(['date_block_num', 'city_id'], as_index=False).item_cnt_day.sum()
ci = ci.rename(columns={'item_cnt_day' : 'target_city'})

all_data = pd.merge(all_data, ci, how='left', on=['date_block_num', 'city_id']).fillna(0)

## Avg. item price and shop revenue trend

In [27]:
#### avg_item_price
aip = sales.groupby('item_id', as_index=False).item_price.mean()
aip = aip.rename(columns={'item_price' : 'avg_item_price'})

all_data = pd.merge(all_data, aip, how='left', on=['item_id'])

#### avg_date_item_price
aip = sales.groupby(['date_block_num', 'item_id'], as_index=False).item_price.mean()
aip = aip.rename(columns={'item_price' : 'avg_date_item_price'})

all_data = pd.merge(all_data, aip, how='left', on=['date_block_num', 'item_id'])

# compute average change of price for the month
all_data['delta_avg_item_price'] = ((all_data['avg_date_item_price'] - all_data['avg_item_price']) / 
                                    all_data['avg_item_price'])
# need to create lag features of delta and drop avg, date, delta columns


#### shop_revenue
aip = sales.groupby(['date_block_num', 'shop_id'], as_index=False).revenue.sum()
aip = aip.rename(columns={'revenue' : 'date_shop_revenue'})

sales = pd.merge(sales, aip, how='left', on=['date_block_num', 'shop_id'])
all_data = pd.merge(all_data, aip, how='left', on=['date_block_num', 'shop_id'])


aip = sales.groupby('shop_id', as_index=False).date_shop_revenue.mean()
aip = aip.rename(columns={'date_shop_revenue' : 'avg_shop_revenue'})

all_data = pd.merge(all_data, aip, how='left', on=['shop_id'])

# compute the change in revenue for the month
all_data['delta_avg_shop_revenue'] = ((all_data['date_shop_revenue'] - all_data['avg_shop_revenue']) / 
                                    all_data['avg_shop_revenue'])
# need to create lag features of delta and drop avg, date, delta columns



#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id', 'item_category_id'],inplace=True)

# Downcast dtypes from 64 to 32 bit to save memory
#all_data = downcast_dtypes(all_data)
gc.collect();

del grid, si, s, i, ic, it, ist, ci, aip, sales, items, item_categories, shops

In [28]:
all_data.head()

,date_block_num,shop_id,item_id,item_category_id,type_id,subtype_id,city_id,target,target_shop,target_item,target_item_cat,target_item_type,target_item_subtype,target_city,avg_item_price,avg_date_item_price,delta_avg_item_price,date_shop_revenue,avg_shop_revenue,delta_avg_shop_revenue
114910,0,2,19,40,11,4,0,0.0,1146.0,1.0,33489.0,43002.0,33489.0,1146.0,28.000000,28.000000,0.000000,1085064.8,1.401849e+06,-0.225976
117150,0,2,27,19,5,10,0,1.0,1146.0,7.0,8983.0,18555.0,10748.0,1146.0,1461.228571,2325.000000,0.591127,1085064.8,1.401849e+06,-0.225976
120623,0,2,28,30,8,55,0,0.0,1146.0,8.0,22216.0,27959.0,22216.0,1146.0,310.010465,549.000000,0.770908,1085064.8,1.401849e+06,-0.225976
118316,0,2,29,23,5,16,0,0.0,1146.0,4.0,4150.0,18555.0,5288.0,1146.0,1759.285714,2397.500000,0.362769,1085064.8,1.401849e+06,-0.225976
114602,0,2,32,40,11,4,0,0.0,1146.0,299.0,33489.0,43002.0,33489.0,1146.0,249.629240,338.110349,0.354450,1085064.8,1.401849e+06,-0.225976


In [29]:
all_data.item_id.dtype, all_data.target_item_cat.dtype

(dtype('int32'), dtype('float64'))

## Mean Encoding:

In [30]:
all_data['target'].sum() / all_data.shape[0]

0.3340756348565541

In [31]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)

all_data['item_target_enc'] = np.NaN

for rest, curr in kf.split(all_data):
    
    # mean encoding item_id
    item_id_target_mean = all_data.iloc[rest].groupby('item_id').target.mean()
    all_data.loc[all_data.index[curr],'item_target_enc'] = all_data['item_id'].map(item_id_target_mean)
    # 0.4164590712798811
    
    # shop_id
    shop_id_target_mean = all_data.iloc[rest].groupby('shop_id').target.mean()
    all_data.loc[all_data.index[curr],'shop_target_enc'] = all_data['shop_id'].map(shop_id_target_mean)
    
    # item_category_id
    item_category_id_target_mean = all_data.iloc[rest].groupby('item_category_id').target.mean()
    all_data.loc[all_data.index[curr],'item_category_target_enc'] = all_data['item_category_id'].map(item_category_id_target_mean)
    
    # type_id
    type_id_target_mean = all_data.iloc[rest].groupby('type_id').target.mean()
    all_data.loc[all_data.index[curr],'type_target_enc'] = all_data['type_id'].map(type_id_target_mean)
    
    # subtype_id
    subtype_id_target_mean = all_data.iloc[rest].groupby('subtype_id').target.mean()
    all_data.loc[all_data.index[curr],'subtype_target_enc'] = all_data['subtype_id'].map(subtype_id_target_mean)
    
    # city_id
    city_id_target_mean = all_data.iloc[rest].groupby('city_id').target.mean()
    all_data.loc[all_data.index[curr],'city_target_enc'] = all_data['city_id'].map(city_id_target_mean)


In [32]:
all_data.item_target_enc.isnull().sum(), all_data.item_target_enc.count()

(578778, 10335026)

In [33]:
# Fill NaNs
all_data.fillna(0.334, inplace=True)

# compute correlation
print(np.corrcoef(all_data['target'].values, all_data['item_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['shop_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['item_category_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['type_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['subtype_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['city_target_enc'].values)[0][1])

0.42497815558405033
0.07165216052324727
0.3471532911951983
0.07037156775355471
0.34729625026395017
0.04779426527548324


In [34]:
all_data.head()

,date_block_num,shop_id,item_id,item_category_id,type_id,subtype_id,city_id,target,target_shop,target_item,target_item_cat,target_item_type,target_item_subtype,target_city,avg_item_price,avg_date_item_price,delta_avg_item_price,date_shop_revenue,avg_shop_revenue,delta_avg_shop_revenue,item_target_enc,shop_target_enc,item_category_target_enc,type_target_enc,subtype_target_enc,city_target_enc
114910,0,2,19,40,11,4,0,0.0,1146.0,1.0,33489.0,43002.0,33489.0,1146.0,28.000000,28.000000,0.000000,1085064.8,1.401849e+06,-0.225976,0.334000,0.142463,0.249496,0.217195,0.249496,0.142463
117150,0,2,27,19,5,10,0,1.0,1146.0,7.0,8983.0,18555.0,10748.0,1146.0,1461.228571,2325.000000,0.591127,1085064.8,1.401849e+06,-0.225976,0.048523,0.142463,0.726472,0.709989,0.673864,0.142463
120623,0,2,28,30,8,55,0,0.0,1146.0,8.0,22216.0,27959.0,22216.0,1146.0,310.010465,549.000000,0.770908,1085064.8,1.401849e+06,-0.225976,0.142424,0.142463,1.189935,0.674091,1.189935,0.142463
118316,0,2,29,23,5,16,0,0.0,1146.0,4.0,4150.0,18555.0,5288.0,1146.0,1759.285714,2397.500000,0.362769,1085064.8,1.401849e+06,-0.225976,0.030303,0.142463,0.681362,0.709989,0.660287,0.142463
114602,0,2,32,40,11,4,0,0.0,1146.0,299.0,33489.0,43002.0,33489.0,1146.0,249.629240,338.110349,0.354450,1085064.8,1.401849e+06,-0.225976,0.894020,0.142463,0.249496,0.217195,0.249496,0.142463


* ***Mean Encoding for test File:***

!!!!!!!!!!!!!!!!! check mean encoding of test done by other peoples

In [35]:
# Adding mean encoding to the test
# mean encoding item_id
item_id_target_mean = all_data.groupby('item_id').target.mean()
test['item_target_enc'] = test['item_id'].map(item_id_target_mean)

# shop_id
shop_id_target_mean = all_data.groupby('shop_id').target.mean()
test['shop_target_enc'] = test['shop_id'].map(shop_id_target_mean)

# item_category_id
item_category_id_target_mean = all_data.groupby('item_category_id').target.mean()
test['item_category_target_enc'] = test['item_category_id'].map(item_category_id_target_mean)

# type_id
type_id_target_mean = all_data.groupby('type_id').target.mean()
test['type_target_enc'] = test['type_id'].map(type_id_target_mean)

# subtype_id
subtype_id_target_mean = all_data.groupby('subtype_id').target.mean()
test['subtype_target_enc'] = test['subtype_id'].map(subtype_id_target_mean)

# city_id
city_id_target_mean = all_data.groupby('city_id').target.mean()
test['city_target_enc'] = test['city_id'].map(city_id_target_mean)

In [36]:
test.head()

,ID,shop_id,item_id,item_category_id,type_id,subtype_id,city_id,item_target_enc,shop_target_enc,item_category_target_enc,type_target_enc,subtype_target_enc,city_target_enc
0,0,5,5037,19,5,10,3,1.950845,0.189383,0.757562,0.712105,0.700825,0.189383
1,1,5,5320,55,13,2,3,NaN,0.189383,0.224159,0.172773,0.224159,0.189383
2,2,5,5233,19,5,10,3,1.656863,0.189383,0.757562,0.712105,0.700825,0.189383
3,3,5,5232,23,5,16,3,1.093023,0.189383,0.665891,0.712105,0.644390,0.189383
4,4,5,5268,20,5,11,3,NaN,0.189383,1.910721,0.712105,1.839340,0.189383


In [37]:
test.item_target_enc.isnull().sum(), test.item_target_enc.count()

(15246, 198954)

In [38]:
# Fill NaNs
test.fillna(0.334, inplace=True)

In [39]:
test.head()

,ID,shop_id,item_id,item_category_id,type_id,subtype_id,city_id,item_target_enc,shop_target_enc,item_category_target_enc,type_target_enc,subtype_target_enc,city_target_enc
0,0,5,5037,19,5,10,3,1.950845,0.189383,0.757562,0.712105,0.700825,0.189383
1,1,5,5320,55,13,2,3,0.334000,0.189383,0.224159,0.172773,0.224159,0.189383
2,2,5,5233,19,5,10,3,1.656863,0.189383,0.757562,0.712105,0.700825,0.189383
3,3,5,5232,23,5,16,3,1.093023,0.189383,0.665891,0.712105,0.644390,0.189383
4,4,5,5268,20,5,11,3,0.334000,0.189383,1.910721,0.712105,1.839340,0.189383


## Adding lag features:

In [40]:
all_data = downcast_dtypes(all_data)

In [41]:
all_data.head()

,date_block_num,shop_id,item_id,item_category_id,type_id,subtype_id,city_id,target,target_shop,target_item,target_item_cat,target_item_type,target_item_subtype,target_city,avg_item_price,avg_date_item_price,delta_avg_item_price,date_shop_revenue,avg_shop_revenue,delta_avg_shop_revenue,item_target_enc,shop_target_enc,item_category_target_enc,type_target_enc,subtype_target_enc,city_target_enc
114910,0,2,19,40,11,4,0,0.0,1146.0,1.0,33504.0,43008.0,33504.0,1146.0,28.000,28.0,0.000000,inf,inf,-0.225952,0.333984,0.142456,0.249512,0.217163,0.249512,0.142456
117150,0,2,27,19,5,10,0,1.0,1146.0,7.0,8984.0,18560.0,10752.0,1146.0,1461.000,2324.0,0.591309,inf,inf,-0.225952,0.048523,0.142456,0.726562,0.709961,0.673828,0.142456
120623,0,2,28,30,8,55,0,0.0,1146.0,8.0,22208.0,27952.0,22208.0,1146.0,310.000,549.0,0.770996,inf,inf,-0.225952,0.142456,0.142456,1.189453,0.674316,1.189453,0.142456
118316,0,2,29,23,5,16,0,0.0,1146.0,4.0,4152.0,18560.0,5288.0,1146.0,1759.000,2398.0,0.362793,inf,inf,-0.225952,0.030304,0.142456,0.681152,0.709961,0.660156,0.142456
114602,0,2,32,40,11,4,0,0.0,1146.0,299.0,33504.0,43008.0,33504.0,1146.0,249.625,338.0,0.354492,inf,inf,-0.225952,0.894043,0.142456,0.249512,0.217163,0.249512,0.142456


In [42]:
%%time
from tqdm import tqdm_notebook

# List of columns that we will use to create lags
lag_cols = ['target', 'target_shop', 'target_item', 'target_item_cat', 'target_item_type', 'target_item_subtype',
            'target_city', 'delta_avg_item_price', 'delta_avg_shop_revenue']
rest_cols = ['date_block_num', 'shop_id', 'item_id']

shift_range = [1, 2, 3, 4, 5, 6]

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[rest_cols + lag_cols].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in lag_cols else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=rest_cols, how='left').fillna(0)

del train_shift


# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]]
# We will drop these at fitting stage
to_drop_cols = [list(set(list(all_data.columns)) - (set(fit_cols)|set(rest_cols))) + ['date_block_num']] 
print('to_drop_cols =', to_drop_cols)

# Category for each item
#item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

#all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')


to_drop_cols = [['delta_avg_shop_revenue', 'target_item_cat', 'target', 'target_city', 'avg_item_price', 'target_item', 'shop_target_enc', 'target_item_type', 'item_category_target_enc', 'type_target_enc', 'avg_shop_revenue', 'avg_date_item_price', 'delta_avg_item_price', 'subtype_id', 'type_id', 'item_target_enc', 'target_shop', 'subtype_target_enc', 'city_target_enc', 'target_item_subtype', 'city_id', 'date_shop_revenue', 'item_category_id', 'date_block_num']]
CPU times: user 1min 23s, sys: 1min 1s, total: 2min 25s
Wall time: 2min 24s


In [43]:
to_drop_cols = ['date_block_num', 'target', 'target_shop', 'target_item', 'target_item_cat', 'target_item_type',
               'target_item_subtype', 'target_city', 'avg_item_price', 'avg_date_item_price', 'delta_avg_item_price',
                'date_shop_revenue', 'avg_shop_revenue', 'delta_avg_shop_revenue']

In [44]:
all_data.head()

,date_block_num,shop_id,item_id,item_category_id,type_id,subtype_id,city_id,target,target_shop,target_item,target_item_cat,target_item_type,target_item_subtype,target_city,avg_item_price,avg_date_item_price,delta_avg_item_price,date_shop_revenue,avg_shop_revenue,delta_avg_shop_revenue,item_target_enc,shop_target_enc,item_category_target_enc,type_target_enc,subtype_target_enc,city_target_enc,target_lag_1,target_shop_lag_1,target_item_lag_1,target_item_cat_lag_1,target_item_type_lag_1,target_item_subtype_lag_1,target_city_lag_1,delta_avg_item_price_lag_1,delta_avg_shop_revenue_lag_1,target_lag_2,target_shop_lag_2,target_item_lag_2,target_item_cat_lag_2,target_item_type_lag_2,target_item_subtype_lag_2,target_city_lag_2,delta_avg_item_price_lag_2,delta_avg_shop_revenue_lag_2,target_lag_3,target_shop_lag_3,target_item_lag_3,target_item_cat_lag_3,target_item_type_lag_3,target_item_subtype_lag_3,target_city_lag_3,delta_avg_item_price_lag_3,delta_avg_shop_revenue_lag_3,target_lag_4,target_shop_lag_4,target_item_lag_4,target_item_cat_lag_4,target_item_type_lag_4,target_item_subtype_lag_4,target_city_lag_4,delta_avg_item_price_lag_4,delta_avg_shop_revenue_lag_4,target_lag_5,target_shop_lag_5,target_item_lag_5,target_item_cat_lag_5,target_item_type_lag_5,target_item_subtype_lag_5,target_city_lag_5,delta_avg_item_price_lag_5,delta_avg_shop_revenue_lag_5,target_lag_6,target_shop_lag_6,target_item_lag_6,target_item_cat_lag_6,target_item_type_lag_6,target_item_subtype_lag_6,target_city_lag_6,delta_avg_item_price_lag_6,delta_avg_shop_revenue_lag_6
0,0,2,19,40,11,4,0,0.0,1146.0,1.0,33504.0,43008.0,33504.0,1146.0,28.000,28.0,0.000000,inf,inf,-0.225952,0.333984,0.142456,0.249512,0.217163,0.249512,0.142456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,27,19,5,10,0,1.0,1146.0,7.0,8984.0,18560.0,10752.0,1146.0,1461.000,2324.0,0.591309,inf,inf,-0.225952,0.048523,0.142456,0.726562,0.709961,0.673828,0.142456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,28,30,8,55,0,0.0,1146.0,8.0,22208.0,27952.0,22208.0,1146.0,310.000,549.0,0.770996,inf,inf,-0.225952,0.142456,0.142456,1.189453,0.674316,1.189453,0.142456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,29,23,5,16,0,0.0,1146.0,4.0,4152.0,18560.0,5288.0,1146.0,1759.000,2398.0,0.362793,inf,inf,-0.225952,0.030304,0.142456,0.681152,0.709961,0.660156,0.142456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,32,40,11,4,0,0.0,1146.0,299.0,33504.0,43008.0,33504.0,1146.0,249.625,338.0,0.354492,inf,inf,-0.225952,0.894043,0.142456,0.249512,0.217163,0.249512,0.142456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* ***Lag Features for test file:***

In [45]:
test.head()

,ID,shop_id,item_id,item_category_id,type_id,subtype_id,city_id,item_target_enc,shop_target_enc,item_category_target_enc,type_target_enc,subtype_target_enc,city_target_enc
0,0,5,5037,19,5,10,3,1.950845,0.189383,0.757562,0.712105,0.700825,0.189383
1,1,5,5320,55,13,2,3,0.334000,0.189383,0.224159,0.172773,0.224159,0.189383
2,2,5,5233,19,5,10,3,1.656863,0.189383,0.757562,0.712105,0.700825,0.189383
3,3,5,5232,23,5,16,3,1.093023,0.189383,0.665891,0.712105,0.644390,0.189383
4,4,5,5268,20,5,11,3,0.334000,0.189383,1.910721,0.712105,1.839340,0.189383


In [46]:
#(rest_cols + lag_cols)[1:]

In [47]:
all_data[all_data.date_block_num == 33][(rest_cols + lag_cols)[1:]].head(1)

,shop_id,item_id,target,target_shop,target_item,target_item_cat,target_item_type,target_item_subtype,target_city,delta_avg_item_price,delta_avg_shop_revenue
10675632,2,30,0.0,727.0,1.0,7680.0,13032.0,7680.0,727.0,-0.601562,-0.101135


In [48]:
# iterate over shift_range
for month_shift in tqdm_notebook(shift_range):
    # for lag_1 last date_block_num targets will be used
    num = (1 - month_shift) + all_data['date_block_num'].max()
    
    # get the target columns for the previous months
    test_shift = all_data[all_data.date_block_num == num][(rest_cols + lag_cols)[1:]].copy()
    
    # change the columns name to the corresponding lag feature
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in lag_cols else x
    test_shift = test_shift.rename(columns=foo)

    test = pd.merge(test, test_shift, on=rest_cols[1:], how='left').fillna(0)

del test_shift

In [49]:
# drop ID column from test
test = test.drop(['ID'], axis=1)

In [50]:
# Downcast dtypes from 64 to 32/16 bit to save memory
all_data = downcast_dtypes(all_data)
test = downcast_dtypes(test)
gc.collect();

In [51]:
#all_data.item_category_target_enc.dtypes, all_data.target_item_cat_lag_4.dtypes

### * Months since the last sale for item and for each shop/item pair

matrix['month'] = matrix['date_block_num'] % 12

days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days).astype(np.int8)


cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num   
        



cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num 

            
            
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

# Memory usage:

In [52]:
all_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10913804 entries, 0 to 10913803
Data columns (total 80 columns):
date_block_num                  int32
shop_id                         int32
item_id                         int32
item_category_id                int8
type_id                         int8
subtype_id                      int8
city_id                         int8
target                          float16
target_shop                     float16
target_item                     float16
target_item_cat                 float16
target_item_type                float16
target_item_subtype             float16
target_city                     float16
avg_item_price                  float16
avg_date_item_price             float16
delta_avg_item_price            float16
date_shop_revenue               float16
avg_shop_revenue                float16
delta_avg_shop_revenue          float16
item_target_enc                 float16
shop_target_enc                 float16
item_category_target_en

In [53]:
#for dtype in ['float','int']:
#    selected_dtype = all_data.select_dtypes(include=[dtype])
#    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
#    mean_usage_mb = mean_usage_b / 1024 ** 2
#    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

In [54]:
# Splitting in train and dev set
to_drop_cols = ['date_block_num', 'target', 'target_shop', 'target_item', 'target_item_cat', 'target_item_type',
               'target_item_subtype', 'target_city', 'avg_item_price', 'avg_date_item_price', 'delta_avg_item_price',
                'date_shop_revenue', 'avg_shop_revenue', 'delta_avg_shop_revenue']


# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 


dates = all_data['date_block_num']

last_block = dates.max()

dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_val =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_val =  all_data.loc[dates == last_block, 'target'].values

# memory usage
X_train.info()
#X_val.info()
#test.info()

del all_data, dates

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6186922 entries, 4488710 to 10675631
Data columns (total 66 columns):
shop_id                         int32
item_id                         int32
item_category_id                int8
type_id                         int8
subtype_id                      int8
city_id                         int8
item_target_enc                 float16
shop_target_enc                 float16
item_category_target_enc        float16
type_target_enc                 float16
subtype_target_enc              float16
city_target_enc                 float16
target_lag_1                    float16
target_shop_lag_1               float16
target_item_lag_1               float16
target_item_cat_lag_1           float16
target_item_type_lag_1          float16
target_item_subtype_lag_1       float16
target_city_lag_1               float16
delta_avg_item_price_lag_1      float16
delta_avg_shop_revenue_lag_1    float16
target_lag_2                    float16
target_shop_lag_

In [55]:
%%time
#all_data.to_csv('all_data.csv')
#test.to_csv('test.csv')

X_train.to_pickle('X_train.gzip')
X_val.to_pickle('X_val.gzip')
y_train.dump('y_train.gzip')
y_val.dump('y_val.gzip')

test.to_pickle('test.gzip')

CPU times: user 676 ms, sys: 2.21 s, total: 2.89 s
Wall time: 2.88 s


* ✔ Need to do sorting or not of all_data => train and predict on this data and merge it on test before submitting
* ✔ add items categories to the final pickle of all_data
* can create features from shops and items name???
* item_price in grid?


In [56]:
#%%time
#pd.read_csv('all_data.csv', index_col=0).head()